In [ ]:
import os
from IPython.display import display, HTML
import pandas as pd
import numpy as np
import pickle
import re
import glob

from main import *
from visualisation import *

import plotly.express as px
import plotly.offline as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=False)
pd.options.plotting.backend = 'plotly'


pd.set_option('display.float_format',  '{:,.2f}'.format)
pd.set_option('display.max_columns', 50 )
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 500)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [ ]:
%%time
data = pd.read_csv(
                    'data/data.csv', 
                   parse_dates=['ts']
                  )

In [ ]:
print(f"{data['ts'].min()}, {data['ts'].max()}, {len(data)}")

In [ ]:
%%time
df, df_hourly, devices_with_faileures = transform_data(data)

In [ ]:
{149:'one month of data with variance', 
 150:'was off only drnnig first month', 
 157:'sevral months of data with a long malnuduncton in the middle', 
 180:'kinda weird - second half only 0.5',
 199:'is on most time', 
 245:'is on all the time', 
 287:'good', 
 296:'good', 
 299:'good', 
 
 93:'good', 158:'qutr of data', 159:'good', 160:'good', 161:'ok', 170:'good', 173:'good', 176:'qutr of data', 177:'qutr of data', 
 229:'off durning second half', 
 284:'good', 293:'good', 426:'always off'
  
}

In [ ]:
# df_hourly.groupby('device_id').agg({'ind_shutdown_mal': ['mean', 'sum','count']}).sort_values(('ind_shutdown_mal', 'mean'), ascending=False)

In [ ]:
status_dict_des = {
    0:'on', ## on
    0.5:'unload',
    1:'off',
    2:'unload',
    3:'off', 
    4:'off',
    5:'unload'
}

status_dict = {0:2, 
               0.5:1,
              1:0,
              2:1,
              3:0,
              4:0}

status_dict_des2 = {
    0:'off', ## on
    1:'energy_consuming',
    2:'compressing'
}


In [ ]:
df.sample(5)

In [ ]:
df['dict_compressor_status'] = df['compressor_status'].map(status_dict)
df['compressing']  = np.where(df['compressor_status']==0, 1, 0)
df['energy_consuming'] = np.where(df['compressor_status'].isin([0, 2, 0.5]), '1', '0')
df['dict_compressor_status_des'] = df['dict_compressor_status'].map(status_dict_des2)




In [ ]:
# pp = df.groupby(['dict_compressor_status_des', 'device_id', ]).agg({'ind_shutdown_mal':['mean', 'count']})
# pp.loc[pp[('ind_shutdown_mal', 'mean')]>0.00001, :]dict_compressor_status

In [ ]:
%%time
df_tmp = df.groupby(['device_id', 'dict_compressor_status_des', 'dict_compressor_status', 'compressing']).resample('2H').agg({'compressor_status':'count', 'ind_shutdown_mal':'max'})
df_tmp2 = pd.DataFrame(df_tmp.groupby(['ts', 'device_id'])[['compressor_status']].max())
sample_df = df_tmp.reset_index().merge(df_tmp2, on=['device_id', 'ts', 'compressor_status']).drop('compressor_status', axis=1).sort_values(['device_id', 'ts'])
sample_df["shutdown_mal"] = sample_df["ind_shutdown_mal"].map({0.00:'normal', 1.00:'mulfunction'})
# sample_df["ind_shutdown_mal"] = sample_df["ind_shutdown_mal"].astype('float')
sample_df

In [ ]:
sample_df#["ind_shutdown_mal"].mean()
sample_df["size_shutdown_mal"] = sample_df["ind_shutdown_mal"].map({0.00:0.05, 1.00:0.07})

In [ ]:
# sample_df.loc[sample_df['ind_shutdown_mal']==1, ['dict_compressor_status_des', 'dict_compressor_status']]# = 'ind_shutdown_mal'
# sample_df['ind_shutdown_mal'] = sample_df['ind_shutdown_mal'].astype('str')

In [ ]:
def ploti_o(df, show):
    df_tmp = df.query('device_id==@show').copy()
    label_mean = df_tmp["ind_shutdown_mal"].mean()
    
#     df_tmp['ind_shutdown_mal'] = df_tmp['ind_shutdown_mal'].astype('str')  
    print([show, label_mean])
#     df_tmp = df_tmp.sample(frac=0.4) 
#     df_tmp.sort_values('ts', inplace=True)
    
    fig = px.scatter(df_tmp, 
                     x='ts', 
                     y='dict_compressor_status_des', 
                     color='shutdown_mal',
#                      size='size_shutdown_mal'
#                      hover_name = 'dict_compressor_status_des'
                    )
    fig.update_layout(title=f'Compressor #{show} with label {label_mean}',
                  yaxis_zeroline=False, xaxis_zeroline=False)
    
    fig.write_html(f"output/scater/scater_{show}.html")
    

In [ ]:
ploti_o(sample_df, '296')

In [ ]:
pp = df.groupby('device_id')['ind_shutdown_mal'].mean().sort_values()
for i in pp.index:
    ploti_o(sample_df, str(i))

In [ ]:
# df_tmp = df.groupby('device_id').resample('2H').agg({'dict_compressor_status':['mode'], 'ind_shutdown_mal':'max'})
pp = df.groupby('device_id').resample('2H')['dict_compressor_status'].apply(pd.Series.mode)
pp

In [ ]:
pp

In [ ]:
df.loc[(df['device_id']==93) & (df.index.month.values==7) & (df.index.day.values==1) & (df.index.hour.values < 8), 'dict_compressor_status']

In [ ]:
df.index.month.values

In [ ]:
df2

In [ ]:
stable_devices = [287, 296, 299, 93, 159, 160, 170, 173, 284, 293]
df_filtered = df_filtered[df_filtered['device_id'].isin(stable_devices)]

In [ ]:
df_tmp = df.groupby('device_id').resample('2H').agg({'active_ind':['sum', 'count'], 'ind_shutdown_mal':'max'})
df_tmp['ratio'] = df_tmp[('active_ind', 'sum')] / df_tmp[('active_ind', 'count')]
df_tmp = df_tmp.reset_index()
df_tmp.columns = ['device_id', 'ts', 'sum_active_ind','count_active_ind', 'max_ind_shutdown_mal', 'ratio']
df_tmp['max_ind_shutdown_mal'] = df_tmp['max_ind_shutdown_mal'].astype('category')

In [ ]:
pp = devices_with_faileures.sort_values(ascending=False).index
for i in df_tmp['device_id'].value_counts().index:
    print(i)
    df_plt = df_tmp.loc[df_tmp['device_id']==i, :]
    fig = px.scatter(df_plt, x='ts', y='ratio', color='max_ind_shutdown_mal')
    fig.write_html(f"output/activaton ratio/activaton ratio_{i}.html")

In [ ]:
fig = px.scatter(df_plt, x='ts', y='ratio', color='max_ind_shutdown_mal')
fig.write_html(f"output/activaton ratio/activaton ratio_{i}.html")

In [ ]:
i = 296
fig = px.scatter(df_tmp.query('device_id==296'), x='ts', y='sum_active_ind', color='max_ind_shutdown_mal')
fig.write_html(f"output/activaton ratio/activaton ratio_{i}.html")

In [ ]:
pp = devices_with_faileures.sort_values(ascending=False).index
for i in pp[:3]:
    print(i)
    fig = px.scatter(df_tmp.query('device_id==@i'), x='ts', y='ratio', color='max_ind_shutdown_mal', hover_name)
    fig.write_html(f"output/activaton ratio/activaton ratio_{i}.html")

In [ ]:
{149:'one month of data with variance', 
 150:'was off only drnnig first month', 
 157:'sevral months of data with a long malnuduncton in the middle', 
 180:'kinda weird - second half only 0.5',
 199:'is on most time', 
 245:'is on all the time', 
 287:'good', 
 296:'good', 
 299:'good', 
 
 93:'good', 158:'qutr of data', 159:'good', 160:'good', 161:'ok', 170:'good', 173:'good', 176:'qutr of data', 177:'qutr of data', 
 229:'off durning second half', 
 284:'good', 293:'good', 426:'always off'
  
}